In [1]:
#import pandas as pd

#data = pd.read_csv('/content/final.csv')
# data['audio_path'] = data['audio_path'].str.replace("\\", "/")

#data['audio_path']= data['audio_path'].str.replace('/content/gdrive/MyDrive/clips/audio','/content/drive/MyDrive/Training Data ASR/audio-20240314T063854Z-001/audio')
#data.to_csv('updated_data.csv', index=False)

In [2]:
#import pandas as pd

#data = pd.read_csv('/content/updated_data.csv')
#data.to_csv('updated_data.csv', index=False, encoding='utf-8')

In [3]:

#data.isnull().sum()

In [4]:

#data.dropna(inplace=True)

In [5]:
#data.duplicated().sum()
#data.isnull().sum()

In [6]:
#data.shape

In [7]:
#import pandas as pd



#data = data.sample(frac=1, random_state=42).reset_index(drop=True)

#train_ratio = 0.8
#test_ratio = 1 - train_ratio

#train_size = int(train_ratio * len(data))

#train_data = data[:train_size]
#test_data = data[train_size:]

#train_data.to_csv('train_data.csv', index=False)
#test_data.to_csv('test_data.csv', index=False)

#print("Training and testing data created successfully.")


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install datasets

In [10]:
from datasets import load_dataset

In [11]:
!pip install transformers

In [12]:
!pip install torch

In [13]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, TrainingArguments, Trainer


In [14]:
common_voice_train = load_dataset("csv", data_files="/content/final_final.csv", split="train")
common_voice_test = load_dataset("csv", data_files="/content/devansh_final_1_2.csv", split="train")

In [15]:
common_voice_train[0]

{'Transcript': 'ਬਾਬਾ ਫਰੀਦ ਨੇ ਰੱਬ ਦੀ ਭਗਤੀ ਕੀਤੀ',
 'text_id': 'A1_001',
 'audio_path': '/content/drive/MyDrive/Training Data ASR/audio-20240314T063854Z-001/audio/A1/A1_001.wav'}

In [16]:
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [17]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [18]:
show_random_elements(common_voice_train.remove_columns(["audio_path"]), num_examples=20)

,Transcript,text_id
0,ਸਾਨੂੰ ਵੱਡਿਆਂ ਦੀ ਗੱਲ ਵਿੱਚ ਦਾਖਲ ਨਹੀਂ ਦੇਣਾ ਚਾਹੀਦਾ,P71_047
1,ਲਾਜੋ ਨੇ ਕੁੱਕੜ ਨੂੰ ਦਾਣੇ ਪਾਏ,A3_407
2,ਸਾਡੇ ਘਰ ਦੇ ਨੇੜੇ ਹੀ ਇੱਕ ਪੀਰ ਦੀ ਸਮਾਧੀ ਹੈ,P34_051
3,ਸਾਹਿਤ ਸਾਹਿਤ ਸਭਾ ਵਿੱਚ ਪਰ ਪਰਸ ਪਰਸਾ ਨਾਵਲ ਦੀ ਕਾਫੀ ਚਰਚਾ ਹੋਈ,P30_121
4,ਮੰਜੀਤ ਨੇ ਕੱਚ ਦੀਆਂ ਵੰਗਾਂ ਖਰੀਦੀਆਂ,A17_080
5,ਕਿਸੇ ਦਾ ਘਰ ਢਾਹੁਣਾ ਚੰਗੀ ਗੱਲ ਨਹੀਂ,A27_243
6,ਭਰਾ ਦੇ ਵਿਆਹ ਤੇ ਨੱਚ ਕੇ ਅਸੀਂ ਬਹੁਤ ਧਮਾਲ ਪਾਈ,A2_215
7,ਕਾਰਪੋਰੇਸ਼ਨ ਵਾਲਿਆਂ ਨੇ ਸਾਰੀਆਂ ਝੋਪੜੀਆਂ ਹਟਾ ਦਿੱਤੀਆਂ,A3_168
8,ਭੰਗੜਾ ਪੰਜਾਬ ਦਾ ਲੋਕ ਨ੍ਰਿਤ ਹੈ,P32_133
9,ਮਹਾਰਾਜਾ ਰਣਜੀਤ ਸਿੰਘ ਬਹੁਤ ਵੱਡੀ ਹਸਤੀ ਸੀ,A4_058


In [19]:
common_voice_train[0]

{'Transcript': 'ਬਾਬਾ ਫਰੀਦ ਨੇ ਰੱਬ ਦੀ ਭਗਤੀ ਕੀਤੀ',
 'text_id': 'A1_001',
 'audio_path': '/content/drive/MyDrive/Training Data ASR/audio-20240314T063854Z-001/audio/A1/A1_001.wav'}

In [20]:
import re

chars_to_ignore_regex = r'[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    transcript = str(batch["Transcript"])  # Convert to string
    cleaned_transcript = re.sub(chars_to_ignore_regex, '', transcript).lower() + " "
    batch["Transcript"] = cleaned_transcript
    return batch


In [21]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [22]:
show_random_elements(common_voice_train.remove_columns(["audio_path"]))

,Transcript,text_id
0,ਹਮੇਸ਼ਾ ਬਚਤ ਦੀ ਆਦਤ ਪਾਓ,P68_119
1,ਮੈਨੂੰ ਜਹਾਜ ਦਾ ਸਫਰ ਬਹੁਤ ਚੰਗਾ ਲੱਗਦਾ ਹੈ,P60_056
2,ਬਹੁਤ ਸਾਰੀਆਂ ਚੀਜਾਂ ਅਸੀਂ ਥੋਕ ਦੇ ਭਾਅ ਲਿਆਉਂਦੇ ਹਾਂ,P40_070
3,ਜਿਮੀਦਾਰ ਕਿਸਾਨਾਂ ਤੋਂ ਲਗਾਨ ਵਸੂਲਦੇ ਸਨ,A29_316
4,ਰਮੇਸ਼ ਕੰਮ ਕਰਨ ਦਾ ਇੱਕ ਬਹਾਨਾ ਲੱਭਦਾ ਰਹਿੰਦਾ ਹੈ,A20_319
5,ਕੋਈ ਵੀ ਕੰਮ ਢੰਗ ਨਾਲ ਕਰੋ,A18_216
6,ਬੱਚੇ ਤੱਪੜਾਂ ਦੇ ਬੈਠ ਕੇ ਪੜਦੇ ਹਨ,A20_416
7,ਰਾਜੂ ਇਸ ਘੱਟਣਾ ਦਾ ਚਸ਼ਮਦੀਦ ਗਵਾਹ ਹੈ,P60_034
8,ਉਹ ਇੱਕ ਉੱਚ ਸੰਸਥਾ ਦਾ ਮੈਂਬਰ ਹੈ,P58_016
9,ਅਸਾਮ ਵਿੱਚ ਬਹੁਤ ਸੰਘਣਾ ਜੰਗਲ ਹੈ,A14_156


In [23]:
def extract_all_chars(batch):
  all_text = " ".join(batch["Transcript"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [24]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/12540 [00:00<?, ? examples/s]

Map:   0%|          | 0/3103 [00:00<?, ? examples/s]

In [25]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [26]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ਉ': 0,
 'ਜ਼': 1,
 'ਓ': 2,
 'ੀ': 3,
 '4': 4,
 'ਪ': 5,
 'ਠ': 6,
 'ੂ': 7,
 'ਧ': 8,
 'ਚ': 9,
 'ੜ': 10,
 '਼': 11,
 'ਐ': 12,
 'ੱ': 13,
 'ਤ': 14,
 'ਟ': 15,
 'ੇ': 16,
 'ਫ਼': 17,
 'ੋ': 18,
 '5': 19,
 'ਏ': 20,
 'ਾ': 21,
 'ਸ': 22,
 'ਢ': 23,
 'ਲ਼': 24,
 'ਈ': 25,
 '1': 26,
 'ਣ': 27,
 'ਨ': 28,
 '2': 29,
 'ਆ': 30,
 'ਹ': 31,
 'ਸ਼': 32,
 'ਵ': 33,
 'ਥ': 34,
 'ਝ': 35,
 'ਗ਼': 36,
 'ਊ': 37,
 'ਔ': 38,
 '_': 39,
 'n': 40,
 'ਖ਼': 41,
 'ਰ': 42,
 '0': 43,
 'ੰ': 44,
 'ਂ': 45,
 '।': 46,
 'ਇ': 47,
 'ਦ': 48,
 "'": 49,
 'ਮ': 50,
 'ੈ': 51,
 'ਡ': 52,
 '6': 53,
 'ਗ': 54,
 'ਭ': 55,
 '3': 56,
 'ਫ': 57,
 ' ': 58,
 'ਬ': 59,
 'ਿ': 60,
 '7': 61,
 'ਅ': 62,
 'p': 63,
 'ਕ': 64,
 '੍': 65,
 'ਲ': 66,
 'ਯ': 67,
 'ਖ': 68,
 'ਘ': 69,
 'ੌ': 70,
 'ਜ': 71,
 'ਛ': 72,
 'ੁ': 73}

In [27]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [28]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

76

In [29]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [30]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [31]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [32]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [33]:
processor.save_pretrained(r"/content/drive/MyDrive/wav2vec2-large-xlsr-pa-IN")


[]

In [34]:
common_voice_train[0]

{'Transcript': 'ਬਾਬਾ ਫਰੀਦ ਨੇ ਰੱਬ ਦੀ ਭਗਤੀ ਕੀਤੀ ',
 'text_id': 'A1_001',
 'audio_path': '/content/drive/MyDrive/Training Data ASR/audio-20240314T063854Z-001/audio/A1/A1_001.wav'}

In [35]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["audio_path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["Transcript"]
    return batch

In [36]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

In [37]:
import librosa
import numpy as np

def resample(batch):
    # Ensure audio data is a NumPy array
    speech = np.asarray(batch["speech"])

    # Check if the original sampling rate is not already 16,000 Hz
    if batch['sampling_rate'] != 16000:
        # Resample the audio
        resampled_speech = librosa.resample(speech, orig_sr=batch['sampling_rate'], target_sr=16000)

        # Update batch with resampled data and sampling rate
        batch["speech"] = resampled_speech
        batch["sampling_rate"] = 16000

    return batch


In [38]:
common_voice_train = common_voice_train.map(resample, num_proc=2)
common_voice_test = common_voice_test.map(resample, num_proc=2)

In [39]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=np.asarray(common_voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [40]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: ਵਿਆਹ ਸਮੇਂ ਕਈ ਰਸਮਾਂ ਨਿਭਾਈਆਂ ਜਾਂਦੀਆਂ ਹਨ 
Input array shape: (70400,)
Sampling rate: 16000


In [41]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [42]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

Map (num_proc=4):   0%|          | 0/12540 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/3103 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [43]:
df1 = common_voice_train.to_pandas()
df1.to_csv('final_voice_train_label1', index=False)


In [44]:
df1 = common_voice_test.to_pandas()
df1.to_csv('final_voice_test_label1', index=False)


In [45]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [46]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [47]:
from datasets import load_metric
wer_metric = load_metric("wer")

<ipython-input-47-7b292a9c8d88>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [48]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [49]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1920: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [51]:
!pip install --upgrade accelerate

In [52]:
!pip install --upgrade transformers

In [53]:
!pip install transformers[torch]

In [54]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=r"/content/drive/MyDrive/wav2vec2-large-xlsr-pa-IN",
  group_by_length=True,
  per_device_train_batch_size=32,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=200,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  load_best_model_at_end=True,  # Load the best checkpoint at the end of training
)

In [55]:
!pip install --upgrade transformers


In [56]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [57]:
!pip show transformers

Name: transformers
Version: 4.39.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [58]:
!nvidia-smi

Thu Mar 28 18:15:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              50W / 400W |   1629MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,5.513200,3.280160,0.999871
1000,1.646600,0.756741,0.592116
1500,0.783300,0.461197,0.412431
2000,0.673300,0.566671,0.423669
2500,0.613500,0.542584,0.403037
3000,0.568000,0.465672,0.402351
3500,0.511000,0.386570,0.327242
4000,0.406800,0.666904,0.370308
4500,0.297200,1.275507,0.486209
5000,0.214900,1.006335,0.418350


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Step,Training Loss,Validation Loss,Wer
500,5.513200,3.280160,0.999871
1000,1.646600,0.756741,0.592116
1500,0.783300,0.461197,0.412431
2000,0.673300,0.566671,0.423669
2500,0.613500,0.542584,0.403037
3000,0.568000,0.465672,0.402351
3500,0.511000,0.386570,0.327242
4000,0.406800,0.666904,0.370308
4500,0.297200,1.275507,0.486209
5000,0.214900,1.006335,0.418350


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call